In [1]:
NODE_COUNT = 4
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/star/fedstellar_DFL_18_07_2024_18_00_37/metrics/participant_"
date_str = "18/07/2024 16:00:37	" # minus two hours for timezone

In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/star/fedstellar_DFL_18_07_2024_18_00_37/metrics/participant_0
   step     value                     timestamp         tag
0   201  0.627584 2024-07-18 16:03:19.001259776  Train/Loss
1   251  0.303562 2024-07-18 16:03:19.585874432  Train/Loss
2   301  0.411731 2024-07-18 16:03:20.150298880  Train/Loss
3   351  0.135263 2024-07-18 16:03:20.707885056  Train/Loss
4   401  0.267515 2024-07-18 16:03:21.266274304  Train/Loss
2024-07-18 16:17:05 2024-07-18 16:00:46
                step     value                     timestamp
tag                                                         
Test/Accuracy   5541  0.892547 2024-07-18 16:16:54.980591104
Test/F1Score    5541  0.872271 2024-07-18 16:16:54.980622336
Test/Loss       5541  0.429058 2024-07-18 16:16:54.980553984
Test/Precision  5541  0.886683 2024-07-18 16:16:54.980602880
Test/Recall     5541  0.892547 2024-07-18 16:16:54.980612864
/home/<User>/repos/csg_is/calculating_scores/DFL/star/fedstell

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,5541,0.892547,2024-07-18 16:16:54.980591104,participant_0
Test/F1Score,5541,0.872271,2024-07-18 16:16:54.980622336,participant_0
Test/Loss,5541,0.429058,2024-07-18 16:16:54.980553984,participant_0
Test/Precision,5541,0.886683,2024-07-18 16:16:54.980602880,participant_0
Test/Recall,5541,0.892547,2024-07-18 16:16:54.980612864,participant_0
Test/Accuracy,11081,0.904771,2024-07-18 16:16:55.017677312,participant_1
Test/F1Score,11081,0.892593,2024-07-18 16:16:55.017710336,participant_1
Test/Loss,11081,0.711192,2024-07-18 16:16:55.017644800,participant_1
Test/Precision,11081,0.907683,2024-07-18 16:16:55.017689344,participant_1


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_43981/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,6927.0,0.9002
Test/F1Score,6927.0,0.8840
Test/Loss,6927.0,0.5771
Test/Precision,6927.0,0.8987
Test/Recall,6927.0,0.9002


In [5]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_43981/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,2769.34,0.01
Test/F1Score,2769.34,0.01
Test/Loss,2769.34,0.19
Test/Precision,2769.34,0.01
Test/Recall,2769.34,0.01


In [6]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_43981/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,6927.0,0.9002
Test/F1Score,6927.0,0.8840
Test/Loss,6927.0,0.5771
Test/Precision,6927.0,0.8987
Test/Recall,6927.0,0.9002
